In [1]:
!pip install ultralytics # Run only when needed.

'pip' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [2]:
import os
import torch
from ultralytics import YOLO
from IPython.display import Image, display
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path

## Load Trained Model and Run Inference

Load the best weights from the completed training run and prepare for inference.


In [3]:
# --- Load Trained Model ---

model_trained = None # Initialize variable
TRAINING_SUCCESS = True
results_dir = "runs/detect/helmet_train18"

# Only proceed if training seemed to complete and results were saved
if TRAINING_SUCCESS and results_dir:
    # Path to the trained model weights
    TRAINED_MODEL_PATH = os.path.join(results_dir, 'weights/best.pt')

    # Check if the trained model file exists
    if not os.path.exists(TRAINED_MODEL_PATH):
        print(f"Error: Trained model not found at {TRAINED_MODEL_PATH}")
        print("Cannot proceed with inference.")
    else:
        # Load the trained model
        print(f"Loading trained model from {TRAINED_MODEL_PATH}")
        try:
            model_trained = YOLO(TRAINED_MODEL_PATH)
            print("Trained model loaded successfully.")
        except Exception as e:
            print(f"An error occurred during trained model loading: {e}")
            model_trained = None # Ensure it's None if loading failed
else:
    print(
        "Skipping trained model loading because training did not complete successfully "
        "or results directory is unknown."
    )

Loading trained model from runs/detect/helmet_train18\weights/best.pt
Trained model loaded successfully.


### Inference on an Image

Use the loaded trained model to run prediction on a sample image.

**Important:** You need to change the `IMAGE_TO_PREDICT` variable in the next cell to the path of an actual image file you want to test.


In [4]:
# --- Inference on an Image ---

# <<< CHANGE THIS TO YOUR IMAGE PATH >>>
IMAGE_TO_PREDICT = 'dataset/test/images/-5-_png_jpg.rf.edf2c1e85be2ab356d95de9f5dac5391.jpg'

if model_trained: # Check if the trained model was loaded successfully
    if not os.path.exists(IMAGE_TO_PREDICT):
        print(f"\nWarning: Test image not found at {IMAGE_TO_PREDICT}")
        print("Skipping image inference. "
              "Please edit the IMAGE_TO_PREDICT variable above with a valid path.")
    else:
        print(f"\nRunning inference on: {IMAGE_TO_PREDICT}")
        try:
            # Detect available device
            if torch.backends.mps.is_available():
                device = 'mps'
                print("Using MPS (Apple Silicon) device")
            elif torch.cuda.is_available():
                device = 'cuda'
                print("Using CUDA device")
            else:
                device = 'cpu'
                print("Using CPU device")


            # Run inference
            predict_results = model_trained.predict(
                source=IMAGE_TO_PREDICT,
                save=True,      # Save image with bounding boxes
                conf=0.1,       # Optional: set confidence threshold
                device=device,
            )
            # Results are saved in `runs/detect/predict*` directory relative to notebook
            print("Prediction results saved.")

            # Optional: Display the saved prediction image
            # Find the latest prediction directory
            predict_save_dir = predict_results[0].save_dir
            img_base_name = os.path.basename(IMAGE_TO_PREDICT)
            predicted_image_path = os.path.join(predict_save_dir, img_base_name)

            if os.path.exists(predicted_image_path):
                print("\nDisplaying Prediction Result:")
                display(Image(filename=predicted_image_path))
            else:
                 print(f"Could not find saved prediction image at {predicted_image_path}")

            # Optional: Process results further if needed
            # for r in predict_results:
            #     print("Boxes detected:")
            #     print(r.boxes.data)
        except Exception as e:
            print(f"An error occurred during inference: {e}")
elif TRAINING_SUCCESS:
     print("Skipping inference because the trained model failed to load.")
else:
    print("Skipping inference because training did not complete successfully.")


Skipping image inference. Please edit the IMAGE_TO_PREDICT variable above with a valid path.


    Running inference on: dataset/test/images/-5-_png_jpg.rf.edf2c1e85be2ab356d95de9f5dac5391.jpg
    Using MPS (Apple Silicon) device
    
    image 1/1 /Users/sam/HKBU_HDDS_YR2_HW/HDDS2302_Applied Deep Learning_Final/project-py/dataset/test/images/-5-_png_jpg.rf.edf2c1e85be2ab356d95de9f5dac5391.jpg: 640x640 2 helmets, 15.7ms
    Speed: 7.5ms preprocess, 15.7ms inference, 47.9ms postprocess per image at shape (1, 3, 640, 640)
    Results saved to [1mruns/detect/predict[0m
    Prediction results saved.
    
    Displaying Prediction Result:



    
![jpeg](execute_model_files/execute_model_5_1.jpg)
    


In [5]:
# Optional: Evaluate Model on Test Set
if model_trained:
    try:
        print("\nEvaluating model on test set...")
        metrics = model_trained.val(data="hf_helmet_dataset/data.yaml", split='test')
        print(f"Model evaluation complete. mAP50-95: {metrics.box.map}")
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")


Evaluating model on test set...
Ultralytics 8.3.112  Python-3.13.3 torch-2.8.0.dev20250420+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 16303MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 797.4294.2 MB/s, size: 47.7 KB)


val: Scanning C:\Users\Administrator\HDDS2302_Helmet\hf_helmet_dataset\labels\test.cache... 2001 images, 27 backgrounds, 0 corrupt: 100%|██████████| 2001/2001 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 126/126 [00:06<00:00, 19.56it/s]


                   all       2001       5518      0.911      0.903      0.944      0.611
                helmet       1626       4480      0.918      0.898      0.944      0.563
             no-helmet        413       1038      0.904      0.908      0.944       0.66
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to c:\Users\Administrator\HDDS2302_Helmet\runs\detect\val3
Model evaluation complete. mAP50-95: 0.6111792025001236
